In [ ]:
from geopy.distance import geodesic
import random

# Definir el punto fijo en el centro de la Ciudad de México
origin = (19.432608, -99.133209)  # Coordenadas aproximadas del Zócalo

# Generar un punto aleatorio a una distancia de 2 km del punto fijo
def random_point_2km_away(center, distance_km):
    angle = random.uniform(0, 360)
    destination = geodesic(kilometers=distance_km).destination(center, angle)
    return (destination.latitude, destination.longitude)

destination = random_point_2km_away(origin, 2)

print("Punto fijo (centro de la Ciudad de México):", origin)
print("Punto aleatorio a 2 km:", destination)

In [ ]:
import osmnx as ox
import geopandas as gpd

# Cargar la red de vías desde el archivo cache_MexicoCity_walk.graphml
graph = ox.load_graphml('cache_MexicoCity_walk.graphml')

# Cargar los datos de crímenes desde el archivo crimenes.geojson
crimenes = gpd.read_file('crimes.geojson')

# Mostrar información básica de los datos cargados
print(graph)
print(crimenes.head())

In [ ]:
# Re-project geometries to a projected CRS (e.g., UTM zone 14N for Mexico City)
crimenes = crimenes.to_crs(epsg=32614)

# Crear buffers de 50 metros alrededor de cada punto
crimenes['buffer'] = crimenes.geometry.buffer(50)

# Mostrar los primeros registros con los buffers
print(crimenes[['weight', 'time_zones', 'buffer']].head())

In [ ]:
def convert_buffers_to_geodataframe():
    """
    Convierte los buffers de crímenes en un GeoDataFrame en formato adecuado para
    encontrar intersecciones con el grafo, conservando todas las propiedades.
    
    Returns:
        gpd.GeoDataFrame: Un GeoDataFrame con los buffers y sus atributos
    """
    # Crear una copia del DataFrame de crímenes
    crime_buffers = gpd.GeoDataFrame(
        {
            'weight': crimenes['weight'],
            'time_zones': crimenes['time_zones'],
            'geometry': crimenes['buffer']
        },
        crs=crimenes.crs
    )

    # Asegurarse de que las geometrías son válidas
    crime_buffers['geometry'] = crime_buffers['geometry'].buffer(0)

    # Verificar que el CRS coincida con el del grafo
    if crime_buffers.crs != graph.graph['crs']:
        crime_buffers = crime_buffers.to_crs(graph.graph['crs'])

    print(f"Se han convertido {len(crime_buffers)} buffers de crímenes")
    print(f"Columnas disponibles: {crime_buffers.columns.tolist()}")

    return crime_buffers

# Ejecutar la función para obtener el GeoDataFrame de buffers
crime_buffers = convert_buffers_to_geodataframe()

In [ ]:
import numpy as np
import geopandas as gpd
from shapely.geometry import Point
import osmnx as ox
from shapely.geometry import Polygon

def mid_point(origin, destination):
        # Calculate the L2 distance between origin and destination
    l2_distance = np.linalg.norm(np.array(origin) - np.array(destination))
    # Calculate the midpoint between origin and destination
    midpoint = ((origin[0] + destination[0]) / 2, (origin[1] + destination[1]) / 2)

    midpoint_point = Point(midpoint[1], midpoint[0])
    midpoint_geom = gpd.GeoSeries([midpoint_point], crs=crimenes.crs)
    buffer_radius = l2_distance

    midpoint_buffer = midpoint_geom.buffer(buffer_radius)

    return midpoint_buffer

def crop_graph(origin, destination, graph):

    midpoint_buffer = mid_point(origin, destination)

    buffer_polygon = midpoint_buffer.geometry[0]

    subgraph = ox.truncate.truncate_graph_polygon(
        graph,
        buffer_polygon,
        truncate_by_edge=True)

    return subgraph

filtered_graph = crop_graph(origin, destination, graph)

In [ ]:
from shapely.geometry import Point
import geopandas as gpd
import networkx as nx
import matplotlib.pyplot as plt

# Versión vectorizada (más rápida para grafos grandes)
def label_nodes_vectorized(graph, buffer_gdf, weight_col='weight'):
    """
    Etiqueta los nodos del grafo con la suma de pesos de buffers que los intersectan
    usando una implementación vectorizada para mejor rendimiento.
    
    Args:
        graph (nx.Graph): Grafo de NetworkX
        buffer_gdf (gpd.GeoDataFrame): GeoDataFrame con buffers y columna de peso
        weight_col (str): Nombre de la columna con pesos en buffer_gdf
        
    Returns:
        nx.Graph: Grafo con atributos 'buffer_weight' añadidos a los nodos
    """
    # Extraer coordenadas de los nodos y crear un GeoDataFrame
    nodes = {node: Point(data['x'], data['y']) for node, data in graph.nodes(data=True)}
    gdf_nodes = gpd.GeoDataFrame(geometry=list(nodes.values()), index=nodes.keys(), crs=buffer_gdf.crs)

    # Spatial join para encontrar qué buffers intersectan con cada nodo
    joined = gpd.sjoin(gdf_nodes, buffer_gdf, how='left')

    # Sumar los pesos de todos los buffers que intersectan con cada nodo
    node_weights = joined.groupby(joined.index)[weight_col].sum().fillna(0)

    # Añadir los pesos como atributos de los nodos
    nx.set_node_attributes(graph, node_weights.to_dict(), name='buffer_weight')
    return graph

def label_nodes_with_buffer_weights(graph, buffer_gdf, weight_col='weight', attribute_name='buffer_weight'):
    """
    Etiqueta nodos del grafo con la suma de pesos de buffers que los intersectan
    usando un índice espacial para mejorar la eficiencia en búsquedas espaciales
    
    Args:
        graph (nx.Graph): Grafo de NetworkX
        buffer_gdf (gpd.GeoDataFrame): GeoDataFrame con buffers y columna de peso
        weight_col (str): Nombre de la columna con pesos en buffer_gdf
        attribute_name (str): Nombre del atributo a añadir a los nodos
        
    Returns:
        nx.Graph: Grafo con atributos actualizados
    """
    # Crear índice espacial para los buffers
    sindex = buffer_gdf.sindex

    # Convertir nodos a GeoDataFrame
    nodes = {node: (data['x'], data['y']) for node, data in graph.nodes(data=True)}
    gdf_nodes = gpd.GeoDataFrame(
        geometry=[Point(x, y) for x, y in nodes.values()],
        index=nodes.keys(),
        crs=buffer_gdf.crs
    )

    # Para cada nodo, encontrar buffers que lo intersectan
    for node_id, node_point in gdf_nodes.geometry.items():
        # Búsqueda espacial con índice
        possible_matches_index = list(sindex.intersection(node_point.bounds))
        possible_matches = buffer_gdf.iloc[possible_matches_index]
        precise_matches = possible_matches[possible_matches.intersects(node_point)]

        # Sumar pesos de buffers intersectados
        total_weight = precise_matches[weight_col].sum()

        # Asignar atributo al nodo
        graph.nodes[node_id][attribute_name] = total_weight

    return graph

# Aplicar la función a tu grafo filtrado
graph_labeled = label_nodes_vectorized(filtered_graph, crime_buffers, weight_col='weight')

# Mapa de calor de pesos usando colormap amarillo-rojo (YlOrRd)
nc = ox.plot.get_node_colors_by_attr(graph_labeled, 'buffer_weight', cmap='YlOrRd')
fig, ax = plt.subplots(figsize=(12, 10))
ox.plot_graph(graph_labeled, node_color=nc, node_size=20, ax=ax)

# Añadir leyenda de color
sm = plt.cm.ScalarMappable(cmap='YlOrRd')
sm.set_array([])
cbar = plt.colorbar(sm, ax=ax, shrink=0.7)
cbar.set_label('Nivel de Peligro (Peso de Crímenes)', size=12)

In [ ]:
from rtree import index
from shapely.geometry import LineString, Point
import networkx as nx
import math
import matplotlib.pyplot as plt
import numpy as np

def custom_weight_strategy(edge_data, node_u_data, node_v_data, buffer_weight):
    """
    Calculate custom edge weight combining length and crime risk factors
    
    Args:
        edge_data: Edge attributes dictionary
        node_u_data: Source node attributes
        node_v_data: Target node attributes
        buffer_weight: Sum of crime buffer weights intersecting the edge
    
    Returns:
        float: Combined risk-aware weight
    """
    base_length = edge_data.get('length', 0)
    # Average buffer weights of the connected nodes
    node_weight = (node_u_data.get('buffer_weight', 0) + node_v_data.get('buffer_weight', 0)) / 2

    # Calculate combined weight - logarithmic scaling to avoid extreme values
    # Higher crime weights will increase the effective "cost" of the edge
    return base_length * (1 + math.log(1 + node_weight + buffer_weight))

def fast_edge_weight_calculation(graph, buffer_gdf, weight_col='weight'):
    """
    Calculate edge weights based on intersecting crime buffers using R-tree for spatial indexing
    
    Args:
        graph: NetworkX graph
        buffer_gdf: GeoDataFrame with crime buffers
        weight_col: Name of the weight column in buffer_gdf
    
    Returns:
        NetworkX graph with updated edge attributes
    """
    print(f"Processing {len(graph.edges())} edges against {len(buffer_gdf)} buffers...")

    # Create spatial index for buffers
    buffer_idx = index.Index()
    buffer_data = []

    # Populate the R-tree index
    for i, (_, row) in enumerate(buffer_gdf.iterrows()):
        buffer_idx.insert(i, row.geometry.bounds)
        buffer_data.append((row.geometry, row[weight_col]))

    # Process each edge
    edge_count = 0
    edges_with_buffers = 0

    for u, v, k, data in graph.edges(data=True, keys=True):
        edge_count += 1

        # Get or create edge geometry
        if 'geometry' in data:
            line = data['geometry']
        else:
            u_geom = Point(graph.nodes[u]['x'], graph.nodes[u]['y'])
            v_geom = Point(graph.nodes[v]['x'], graph.nodes[v]['y'])
            line = LineString([u_geom, v_geom])

        # Find potential buffer intersections using R-tree
        buffer_ids = list(buffer_idx.intersection(line.bounds))
        total_buffer_weight = 0
        buffer_count = 0

        # Check for precise intersections
        for buf_id in buffer_ids:
            geom, weight = buffer_data[buf_id]
            if line.intersects(geom):
                total_buffer_weight += weight
                buffer_count += 1

        if buffer_count > 0:
            edges_with_buffers += 1

        # Get node data
        node_u_data = graph.nodes[u]
        node_v_data = graph.nodes[v]

        # Calculate edge weight using custom strategy
        data['combined_weight'] = custom_weight_strategy(
            data, node_u_data, node_v_data, total_buffer_weight
        )

        # Store metrics for analysis
        data['buffer_count'] = buffer_count
        data['buffer_influence'] = total_buffer_weight

    print(f"Processed {edge_count} edges, {edges_with_buffers} have buffer intersections")
    return graph

# Apply edge weight calculation
filtered_graph = fast_edge_weight_calculation(
    filtered_graph,
    crime_buffers,
    weight_col='weight'
)

# Visualization with improved color mapping
fig, ax = plt.subplots(figsize=(10, 10))

# Create better color scale with Matplotlib's viridis colormap
ec = ox.plot.get_edge_colors_by_attr(
    filtered_graph,
    'combined_weight',
    cmap='plasma',
    num_bins=10
)

# Plot graph with edge colors based on combined weight
ox.plot_graph(
    filtered_graph,
    edge_color=ec,
    edge_linewidth=1.5,
    node_size=0,
    ax=ax,
    bgcolor='white'
)

# Add color bar
sm = plt.cm.ScalarMappable(cmap='plasma')
sm.set_array([])
plt.colorbar(sm, ax=ax, label='Combined Risk Weight', shrink=0.7, pad=0.02)

In [ ]:
import folium
from folium.plugins import MarkerCluster
import matplotlib.pyplot as plt
import numpy as np

# Create a base map centered on Mexico City
map_center = [19.38, -99.15]  # Approximate center of Mexico City
crime_map = folium.Map(location=map_center, zoom_start=12, tiles='OpenStreetMap')

# Create a custom colormap for gradient colors - yellow to red
cmap = plt.cm.YlOrRd  # Yellow to Red colormap

# Create a copy of the crimenes dataframe and convert it to WGS84 (EPSG:4326)
crimenes_wgs84 = crimenes.copy()
crimenes_wgs84 = crimenes_wgs84.to_crs(epsg=4326)

# Convert the buffer geometries to WGS84
buffer_gdf = gpd.GeoDataFrame(crimenes[['weight']], geometry=crimenes['buffer'], crs=crimenes.crs)
buffer_gdf = buffer_gdf.to_crs(epsg=4326)

# Normalize weights to ensure they fall in the 0-1 range
# (in case they're not already normalized)
max_weight = crimenes['weight'].max()
min_weight = crimenes['weight'].min()
weight_range = max_weight - min_weight

# Add crime buffers to the map
for idx, row in buffer_gdf.iterrows():
    # Create a popup with crime information
    popup_text = f"""
    <b>Weight:</b> {crimenes.iloc[idx]['weight']}<br>
    """

    # Normalize weight if needed, or use directly if already in 0-1 range
    if weight_range > 0 and (max_weight > 1 or min_weight < 0):
        weight_value = (crimenes.iloc[idx]['weight'] - min_weight) / weight_range
    else:
        weight_value = crimenes.iloc[idx]['weight']

    # Use the weight to determine color from colormap
    rgba_color = cmap(weight_value)
    hex_color = '#{:02x}{:02x}{:02x}'.format(
        int(rgba_color[0]*255),
        int(rgba_color[1]*255),
        int(rgba_color[2]*255)
    )

    # Add the buffer as a GeoJson polygon
    folium.GeoJson(
        row.geometry,
        style_function=lambda x, color=hex_color: {
            'fillColor': color,
            'color': color,
            'weight': 1,
            'fillOpacity': 0.25  # Increased from 0.15 for better visibility
        },
        popup=folium.Popup(popup_text, max_width=300)
    ).add_to(crime_map)

# Add a more accurate legend with gradient colors matching the colormap
gradient_legend = """
<div style="position: fixed; 
     bottom: 50px; right: 50px; width: 200px; 
     border:2px solid grey; z-index:9999; font-size:14px;
     background-color: white; padding: 10px;
     ">
     <b>Crime Weight</b><br>
     <div style="background: linear-gradient(to right, #ffff00, #ff0000); 
          height: 20px; margin-top: 5px; margin-bottom: 5px;">
     </div>
     <span style="float: left;">Low</span>
     <span style="float: right;">High</span>
     <div style="clear: both;"></div>
</div>
"""

crime_map.get_root().html.add_child(folium.Element(gradient_legend))

# Add title
title_html = '''
<h3 align="center" style="font-size:16px"><b>Crime Buffer Zones Map</b></h3>
'''
crime_map.get_root().html.add_child(folium.Element(title_html))

In [ ]:
def dijkstra_custom(graph, orig_point, dest_point):
    orig_node = ox.nearest_nodes(graph, orig_point[1], orig_point[0])
    dest_node = ox.nearest_nodes(graph, dest_point[1], dest_point[0])

    return nx.shortest_path(
        graph,
        source=orig_node,
        target=dest_node,
        weight='combined_weight',
        method='dijkstra'
    )

x = dijkstra_custom(filtered_graph, origin, destination)
# Calculate route length and get coordinates
route_length = sum(filtered_graph.edges[u, v, 0]['length'] for u, v in zip(x[:-1], x[1:]))
route_risk = sum(filtered_graph.edges[u, v, 0].get('buffer_influence', 0) for u, v in zip(x[:-1], x[1:]))

# Extract route coordinates for visualization
route_coords = [(filtered_graph.nodes[node]['y'], filtered_graph.nodes[node]['x']) for node in x]

# Visualize the custom weighted route
custom_route_line = ox.plot_graph_route(filtered_graph, x, route_color='green', route_linewidth=6, node_size=0)

print(f"Route based on custom weights:")
print(f"Total length: {route_length:.2f} meters")
print(f"Total risk score: {route_risk:.2f}")

In [ ]:
origin_node = ox.distance.nearest_nodes(filtered_graph, origin[1], origin[0])
destination_node = ox.distance.nearest_nodes(filtered_graph, destination[1], destination[0])

# Calculate the route using combined_weight
shortest_route = nx.shortest_path(filtered_graph, origin_node, destination_node, weight='length')

secure_route = nx.shortest_path(filtered_graph, origin_node, destination_node, weight='combined_weight')

# Extract the coordinates for visualization
route_coords = [(filtered_graph.nodes[node]['y'], filtered_graph.nodes[node]['x']) for node in shortest_route]

# Visualize the route
fig, ax = ox.plot_graph_route(filtered_graph, shortest_route, route_color='red', route_linewidth=6, node_size=0)

fig, ax = ox.plot_graph_route(filtered_graph, secure_route, route_color='blue', route_linewidth=6, node_size=0)

In [ ]:
import networkx as nx

origin_node = ox.distance.nearest_nodes(filtered_graph, origin[1], origin[0])
destination_node = ox.distance.nearest_nodes(filtered_graph, destination[1], destination[0])

# Calculate the route using combined_weight
shortest_route = nx.shortest_path(filtered_graph, origin_node, destination_node, weight='length')

safest_route = nx.shortest_path(filtered_graph, origin_node, destination_node, weight='combined_weight')

# Extract the coordinates for visualization
route_coords = [(filtered_graph.nodes[node]['y'], filtered_graph.nodes[node]['x']) for node in shortest_route]
safest_route_coords = [(filtered_graph.nodes[node]['y'], filtered_graph.nodes[node]['x']) for node in safest_route]

# Add the routes to the map
folium.PolyLine(route_coords, color='blue', weight=4, opacity=0.7).add_to(crime_map)
folium.PolyLine(safest_route_coords, color='green', weight=4, opacity=0.7).add_to(crime_map)

# Mostrar el mapa con la ruta
crime_map